<a href="https://colab.research.google.com/github/Adrianxwu/MDO-ML-IVC-ITB-2021/blob/main/Copy_of_Tutorial_on_Stochastic_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TUTORIAL 2: STOCHASTIC OPTIMIZATION USING EVOLUTIONARY ALGORITHM**

**Multidisciplinary Optimization and Machine Learning for Engineering Design International Virtual Course**

**19 July - 5 August 2021**

Written by: Yohanes Bimo Dwianto

Based on the lecture material by: Hemant Kumar Singh, Ph.D. (University of New South Wales Canberra, Australia)


---
---
# 1. Theory Overview

---
## General Form of Optimization
Suppose that we want to do a minimization problem, the general form of an optimization problem can be written as follows:

\begin{equation} 
\begin{array}{ll}
\mbox{Minimize}     & f_i(\boldsymbol{x}),\ i=1,\ldots,N_{obj} \\
\mbox{subject to}   & g_j(\boldsymbol{x}) \le 0,\ j=1,\ldots,N_{ineq}\\
& h_k(\boldsymbol{x}) = 0,\ k=1,\ldots,N_{eq}\\
& \boldsymbol{x}_L \le \boldsymbol{x} \le \boldsymbol{x}_U
\end{array}    
\end{equation}

where 
* $\boldsymbol{x}$ denotes the $N_{var}$-dimensional solution vector
* $f_i$, $g_j$, and $h_k$ denote the $i^{th}$ objective function, the $j^{th}$ inequality constraint, and the $k^{th}$ equality constraint, respectively
* $N_{obj}$ denotes the number of objective functions
* $N_{ineq}$ and $N_{eq}$ denote the number of inequality constraints and equality constraints, respectively. Then, the number of all constraints is $N_{con}=N_{ineq}+N_{eq}$
* Both $\boldsymbol{x}_L$ and $\boldsymbol{x}_U$ denote the lower and upper bounds of the search space, respectively. 
* In this tutorial, we will focus only on **single-objective optimization ($N_{obj}=1$)**.
---
## Stochastic Optimization
* Stochastic optimization can be defined as minimizing or maximizing a problem with the help of randomness.
* Statistical analysis is needed to ensure its performance.
* Some popular methods: evolutionary algorithm (EA), particle swarm optimization, ant colony optimization, etc.
---
## Evolutionary Algorithm: General Framework
* Population initialization
* Calculation of fitness values, $F$
* Parental selection
* Crossover/ recombination
* Mutation
* Environmental selection

![picture](https://drive.google.com/uc?export=view&id=1T8HdBIVAHh_1-GRW28ABp0TiZN8EZeuR)

In the case of **unconstrained** optimization, the **fitness value** of each individual, $F(\boldsymbol{x})$, **equals** its **objective value**.

In the case of **constrained** optimization, the **fitness value** of each individual, $F(\boldsymbol{x})$, is a **combination of its objective and constraint violation values**; one example is described in the next section.

---
## Constrained Handling Technique (CHT)
* EA was initially designed to solve unconstrained problems. However, various constrained EAs have been developed to handle problems with constraints. To this end, a constraint handling technique needs to be implemented in order to solve constrained problems. In this tutorial, we will give you a simple constraint handling technique so that you can appreciate the main idea of constrained EA. 
* In EA, equality constraints are often modified into inequality constraints, reads as follows
\begin{equation}
g_{N_{ineq}+k}(\boldsymbol{x}) = \lvert h_k(\boldsymbol{x}) \rvert-\epsilon \le 0, \ k=1,\ldots,N_{eq}
\end{equation}
where $\epsilon$ is a user-defined small number, typically $\epsilon=1 \times 10^{-4}$.

* Let's talk about **Penalty function**, which is one of the simplest and probably the most popular CHT technique. A penalty function is generally expressed as follows
\begin{equation} 
    F(\boldsymbol{x})= f_r(\boldsymbol{x}) + p(\boldsymbol{x})
\end{equation}
where $f_r(\boldsymbol{x})$ is the search in objective space and $p(\boldsymbol{x})$ is the penalty function, representing the search in constraint space, usually based on constraint violation value, $\nu_i(\boldsymbol{x})$,
\begin{equation} 
	\nu_i(\boldsymbol{x}) = \max(0,g_i(\boldsymbol{x}))
\end{equation}
---
## Statistical Analysis: Box Plot
* Due to its random nature, multiple runs need to be conducted in EA to ensure its performance. This is particularly useful in the context of research when you want to compare various EA algorithms. Typically, you want to compare the value of the best solution observed during one run of EA,
* We can use boxplot for visualizing the performance of EA on multiple independent runs. A box plot visualizes the optimum objective values obtained by all EA runs in statistical form, as shown in the figure below.

![picture](https://drive.google.com/uc?export=view&id=1ZaPBo4W9UB2xK_fGN9A4RgnDsd1CypFX)

* "maximum", median, and "minimum" values are the most common values to be considered in the performance analysis.

* When comparing algorithms, it is highly recommended to use **statistical hypothesis test** (e.g. Mann-Whitney test) to check whether an algorithm is significantly better than other algorithms or not.

---
---
# 2. Python Coding of Real-coded Genetic Algorithm (Unconstrained and Constrained Optimization)
In this tutorial, we will use Real-coded Genetic Algorithm (RGA), which employs real values to encode the design variables. Here's what you will need to define first:
## Problem Inputs:
* Objective function
* Constraint function(s), if any
* Number of design variables, $N_{var}$
* Number of constraints, $N_{con}$
* Search space: lower bound $lb$ and upper bound $ub$

## RGA Inputs
* Population size, $N_{pop}$
* Number of generations, $N_{gen}$
* Crossover probability, $P_{cross}$
* Mutation probability, $P_{mut}$ 



---
## Importing necessary packages

We will code a simple GA from stratch! So you only need to import ```numpy``` and ```matplotlib``` (for visualization):

In [ ]:
#@title Importing Packages
import numpy as np
import matplotlib.pyplot as plt

---
## Defining Test Problem: 

**Unconstrained Problem: Ackley Problem**

Let's consider a very difficult problem, namely the **Ackley** problem. Because of its multimodality (i.e. many local optima), the Ackley problem serves as a good test problem for demonstrating the strenght of EA in solving multimodal problems. See the visualization below:

![picture](https://drive.google.com/uc?export=view&id=1CgmQ7cMcswJBi7By4nepoB1JwsGAkM0g)

It has the following characteristics:
* Ackley has number of design variables, $N_{var}$, which may vary from one to infinity. 
* This problem has a minimum objective value of zero, located in $\boldsymbol{x}=\{0,\ldots,0\}$.
* It is expressed as follows:
$$f(\boldsymbol{x})=-a \exp{ \left(-b\sqrt{\frac{1}{N_{var}}\sum^{N_{var}}_{i=1}x^{2}_{i}} \right)}-\exp{\left( \frac{1}{N_{var}} \sum^{N_{var}}_{i=1} \cos{\left( cx_i \right)}\right)}+a+\exp{\left( 1 \right)}$$ 
where $a=20$, $b=0.2$, and $c=2\pi$. 
* The function is evaluated at the hypercube $x_{i} \in [-32.768, 32.768]$, for all $i=1,\ldots,N_{var}$

**Reference**:
https://www.sfu.ca/~ssurjano/ackley.html
<br><br>
**Constrained problem: Welded Beam** 
Our second test problem is the constrained **welded beam** problem, which creates extra challenges for EAs because we need CHT to find the feasible optimal solution. See the illustration of the welded beam problem below:

![picture](https://drive.google.com/uc?export=view&id=157ZHCeL7ndL1virle5u72Tl9tyMEBp98)

The welded beam problem has the following characteristics:
* Welded Beam problem has four design variables, $N_{var}=4$ and six constraints, $N_{con}=6$ (all inequality constraints). 
* The currently found minimum feasible objective value is $f(\boldsymbol{x}_{opt})=1.724855673$, located in $\boldsymbol{x}_{opt}=\{0.20573,3.470489,9.036624,0.20573\}^T$
* The objective function is to minimize:
$$f(\boldsymbol{x})=1.10471x_1^2x_2+0.04811x_3x_4(14.0+x_2)$$
subject to:
$$g_1(\boldsymbol{x})=\tau(\boldsymbol{x})-\tau_{max} \le 0$$
$$g_2(\boldsymbol{x})=\sigma(\boldsymbol{x})-\sigma_{max} \le 0$$
$$g_3(\boldsymbol{x})=x_1-x_4 \le 0$$
$$g_4(\boldsymbol{x})=0.10471x_1^2+0.04811x_3x_4(14.0+x_2)-5 \le 0$$
$$g_5(\boldsymbol{x})=\delta(\boldsymbol{x})-\delta_{max} \le 0$$
$$g_6(\boldsymbol{x})=P-P_c(\boldsymbol{x} \le 0$$
where
$$\tau(\boldsymbol{x})=\sqrt{(\tau')^2+2\tau'\tau''\frac{x_2}{2R}+(\tau'')^2}$$
$$\tau'=\frac{P}{\sqrt{2}x_1x_2},\tau''=\frac{MR}{J},M=P\left( L+\frac{x_2}{2}\right)$$
$$R=\sqrt{\frac{x_2^2}{4}+\left( \frac{x_1+x_3}{2}\right)^2}$$
$$J=2\left(\sqrt{2}x_1x_2\left[ \frac{x_2^2}{12}+\left( \frac{x_1+x_3}{2}\right)^2\right]\right)$$
$$\sigma(\boldsymbol{x})=\frac{6PL}{x_4x_3^2},\delta(\boldsymbol{x})=\frac{4PL}{x_3x_4^2}$$
$$P_c(\boldsymbol{x})=\frac{4.013E\sqrt{\frac{x_3^2x_4^6}{36}}}{L^2}\left( 1-\frac{x_3}{2L}\sqrt{\frac{E}{4G}} \right)$$
$$P=6000 \ lb, L=14 \ in, E=30 \times 10^6 \ psi, G=12 \times 10^6 \ psi$$
$$\tau_{max}=13600 \ psi, \sigma_{max}=30000 \ psi, \delta_{max}=0.25 \ in$$
<br><br>
**Reference**:
Rao, Singiresu S. Engineering optimization: theory and practice. John Wiley & Sons, 2019.

Let's define the two problems by executing the following cell:

In [ ]:
#@title Test Problems
def testproblem(indi_denorm,ncon,fun_name):
    nvar = indi_denorm.shape[0] # number of design variables

    if fun_name == 'ackley':
        a = 20
        b = 0.2
        c = 2*np.pi
        
        f = -a*np.exp(-b*np.sqrt(1/nvar*np.sum(indi_denorm**2))) - \
            np.exp(1/nvar*np.sum(np.cos(c*indi_denorm))) + a + np.exp(1)

    elif fun_name == 'weldedbeam':

        E = 30e6
        G = 12e6
        L = 14
        taw_max = 13600
        sigma_max = 30000
        delta_max = 0.25
        P = 6000
        c1 = 0.10471
        c2 = 0.04811
        c3 = 1

        Vweld = indi_denorm[0]**2*indi_denorm[1]
        Vbar = indi_denorm[2]*indi_denorm[3]*(L+indi_denorm[1])

        taw_d = P/(np.sqrt(2)*indi_denorm[0]*indi_denorm[1])
        M = P*(L+indi_denorm[1]/2)
        R = np.sqrt(indi_denorm[1]**2/4+((indi_denorm[0]+indi_denorm[2])/2)**2)
        J = 2*(indi_denorm[0]*indi_denorm[1]*np.sqrt(2)*(indi_denorm[1]**2/12 + ((indi_denorm[0]+indi_denorm[2])/2)**2))
        taw_dd = M*R/J
        taw = np.sqrt(taw_d**2 + 2*taw_d*taw_dd*indi_denorm[1]/(2*R) + taw_dd**2)
        sigma = 6*P*L/(indi_denorm[3]*indi_denorm[2]**2)
        delta = 4*P*L**3/(E*indi_denorm[3]*indi_denorm[2]**3)
        Pc = 4.013*E*indi_denorm[2]*indi_denorm[3]**3/(6*L**2)*(1-indi_denorm[2]/(2*L)*np.sqrt(E/(4*G)))

        f = (c1+c3)*Vweld + c2*Vbar
        g = np.zeros(ncon)
        g[0] = taw - taw_max
        g[1] = sigma - sigma_max
        g[2] = indi_denorm[0] - indi_denorm[3]
        g[3] = P - Pc
        g[4] = delta - delta_max
        g[5] = c1*indi_denorm[0]**2 + c2*indi_denorm[2]*indi_denorm[3]*(L+indi_denorm[1]) - 5

    if ncon == 0:
        return f
    else:
        return f,g

# TESTING - ACKLEY PROBLEM   
nvar = 2
ncon = 0
x = np.array([0,0])
fun_name = 'ackley'
f = testproblem(x,ncon,fun_name)
print('ACKLEY PROBLEM')
print('Optimum objective value:',f)

# TESTING - WELDED BEAM PROBLEM   
#lb = np.array([0.125,0.1,0.1,0.1])
#ub = np.array([2,10,10,2])
nvar = 4
ncon = 6
x = np.array([0.20573,3.470489,9.036624,0.20573])
fun_name = 'weldedbeam'
f = testproblem(x,ncon,fun_name)
print('\nWELDED BEAM PROBLEM')
print('Currently obtained optimum objective:',f[0])
print('Currently obtained optimum constraint:',f[1])


ACKLEY PROBLEM
Optimum objective value: 4.440892098500626e-16

WELDED BEAM PROBLEM
Currently obtained optimum objective: 1.7248556738155942
Currently obtained optimum constraint: [-0.02539959 -0.05312238  0.         -0.03155555 -0.23554035 -3.43298099]


---
## Population Initialization
The initial population can be generated either randomly or by using design of experiment (DoE) techniques such as Latin Hypercube sampling (this is the most common), Sobol sequences, Halton sequences, etc. The parameters that we define are as follows:
* population size, $N_{pop}$ and
* number of design variables, $N_{var}$.

It is common to generate population in normalized values between zero and one.

The cell below demonstrates a simple code to randomly generate initial population (if you set ```nvar=2``` then you can visualize the initial population):


In [ ]:
#@title Population Initialization
# Population Initialization
npop = 10
nvar = 2
population = np.random.rand(npop,nvar)
print(population)

if nvar == 2:
    fig = plt.figure()
    plt.scatter(population[:,0],population[:,1])
    plt.xlabel(r'$x_1$')
    plt.ylabel(r'$x_2$')
    plt.show()

---
## Denormalization
Since the population is defined in a normalized value from zero to one, each variable of the individual in the population needs to be denormalized first into the real values of design variables with the following expression
$$x_{j,denorm} = x_{j,L} + (x_{j,U}-x_{j,L}) x_{j,norm}$$

We use the below function to perform the denormalization operation by using the lower and the upper bound of the Ackley function as an example:

In [ ]:
#@title Denormalization
def calc_denorm(individual,lb,ub):
	indi_denorm = lb + (ub-lb)*individual
	return indi_denorm

lb = -32.768*np.ones(nvar) # lower bound
ub = 32.768*np.ones(nvar)  # upper bound
x = population[0,:]
print('Normalized x:',x)
x1 = calc_denorm(x,lb,ub)
print('Denormalized x:',x1)


---
## Calculating Objective and Constraint function values
The objective and constraint function values are then evaluated based on the defined optimization problem. In the Ackley problem, only the objective function value is evaluated since no constraint is involved. Let's do a simple test, that is, we will evaluate the objective and constraint values of a random initial population for the Ackley and the welded beam problem:

In [ ]:
#@title Calculating Objective and Constraint
def calc_obj_con(population,fun_name,ncon,lb,ub,cht_type=None):
    npop,nvar = population.shape
    objval = np.zeros(npop)         # initializing storing array for objective values
    if ncon == 0:
        conval = np.zeros((npop,1)) # dummy constraint for unconstrained problem
    else:
        conval = np.zeros((npop,ncon)) # initializing storing array for constraint values
      
    for i in range(npop):
        individual = population[i,:]
        indi_denorm = calc_denorm(individual,lb,ub)
        if ncon == 0:
           objval[i] = testproblem(indi_denorm,ncon,fun_name)
        else:
           objval[i],conval[i,:] = testproblem(indi_denorm,ncon,fun_name)

    return objval,conval

# Testing ACKLEY Problem
fun_name = 'ackley'
nvar = 2
ncon = 0
lb = -32.768*np.ones(nvar) # lower bound
ub = 32.768*np.ones(nvar)  # upper bound
npop = 10
population = np.random.rand(npop,nvar)
objval,conval = calc_obj_con(population,fun_name,ncon,lb,ub,cht_type=None)

print('ACKLEY PROBLEM')
print('objective values:',objval)
print('constraint values:',conval)

# Testing ACKLEY Problem
fun_name = 'weldedbeam'
nvar = 4
ncon = 6
lb = np.array([0.125,0.1,0.1,0.1])
ub = np.array([2,10,10,2])
npop = 10
population = np.random.rand(npop,nvar)
objval,conval = calc_obj_con(population,fun_name,ncon,lb,ub,cht_type='SoF')

print('\nWELDED BEAM PROBLEM')
print('objective values:',objval)
print('constraint values:',conval)


---
## Calculating Fitness value from Objective and Constraint Values
So you just calculated the objectives and constraint value for a random population, what you need to do next is to evaluate the **fitness value** for all the individuals in the population:

* The objective and constraint values of each individual in the population are converted into **fitness value** to be assessed in EA. 
* In the case of **unconstrained problem** such as the Ackley problem, **the fitness value of each individual equals its objective value**.
* In the case of constrained problems, the conversion of objective and constraint values to the fitness value primarily depends on the constraint handling technique (CHT). In this tutorial, two CHTs are introduced, namely:

1.   the superiority of feasible individual (SoF) [Deb, 2000]: a CHT technique that highly favors feasible individuals
2.   the generalized multiple constraint ranking (G-MCR) [Dwianto, 2020]: a CHT technique that balances the search between feasible and infeasible regions

**SoF** follows the following expression:
$$
    F(\boldsymbol{x})=\left\{
        \begin{array}{ll}
            f(\boldsymbol{x}), & \text{if } CV(\boldsymbol{x})=0, \\
            f_{worst} + CV(\boldsymbol{x}), & \text{otherwise}.
        \end{array}
    \right.
$$
where $f_{worst}$ is the worst objective value of feasible individual in the current population. In case there is no feasible individual, $f_{worst}$ is set to zero. $CV(\boldsymbol{x})$ is the sum of constraint violations, i.e., $CV(\boldsymbol{x})=\sum_i^{N_{con}} \nu_i(\boldsymbol{x})$

while **G-MCR** follows
\begin{equation} 
    F=\beta_1 R_f+\beta_2 (\eta R_{N_\nu} + \gamma \sum_{i=1}^{m} \alpha_i R_{\nu_i})
\end{equation}
where $R$ denotes the rank of the corresponding subscript in a population. The subscripts $f$, $N_\nu$, and $\nu_i$ indicate the objective value, number of constraints violated, and the $i^{th}$ constraint violation ($\nu_i=\max(0,g_i)$). 
$$\beta_1=\sqrt{1-(\zeta-1)^2},\ \beta_2=1-\beta_1,\ \zeta \equiv \frac{N_{feas}}{N_{pop}}$$ 
$$\alpha_i=\frac{N_{viol_i}}{N_{pop}},\ \gamma=\frac{1}{\sum_i^{m} \alpha_i},\ \eta=0$$.
<br><br>
**REFERENCES**
* Deb, Kalyanmoy. "An efficient constraint handling method for genetic algorithms." Computer methods in applied mechanics and engineering 186, no. 2-4 (2000): 311-338.
* Dwianto, Yohanes Bimo, Hiroaki Fukumoto, and Akira Oyama. "Adaptively preserving solutions in both feasible and infeasible regions on generalized multiple constraint ranking." In Proceedings of the 2020 Genetic and Evolutionary Computation Conference, pp. 690-698. 2020.

The cell below defines our CHT techniques:

In [ ]:
#@title Calculating Fitness
def evaluate_fitval(objval,conval,cht_type=None):
    npop,ncon = conval.shape
    fitval = np.zeros(npop)

    # Calculating constraint violation fron constraint values
    cv = conval.copy()
    cv[cv<0] = 0

    Nv = np.sum(cv>0,axis=1,dtype='int')  # counting number of constraints violated for each individual
    countfeas = np.sum(Nv==0)             # counting the number of feasible individuals

    if cht_type is None:
        fitval[:] = objval
    elif cht_type == 'SoF':
        sumcv = np.sum(cv,axis=1) # sum of constraint violations
        if countfeas > 0:
            objmax = np.amax(objval[np.where(Nv==0)[0]])
        else:
            objmax = 0
        for i in range(npop):
            if sumcv[i] == 0.0:
                fitval[i] = objval[i]
            else:
                fitval[i] = objmax + sumcv[i]
    elif cht_type == 'G-MCR':
        con_check = np.sum(cv>0,axis=0)
        Fobj = np.zeros(npop)
        Fcon = np.zeros((npop,ncon))
        FNv = np.zeros(npop)
        for i in range(npop):
            Fobj[i] = np.sum(objval<objval[i])
            FNv[i] = np.sum(Nv<Nv[i])
            for j in range(ncon):
                Fcon[i,j] = np.sum(cv[:,j]<cv[i,j])

        beta1 = np.sqrt(1-(countfeas/npop-1)**2)
        beta2 = 1-beta1
        eta = 0
        alpha = (1/npop)*con_check
        sum_alpha = np.sum(alpha)			
        if sum_alpha > 0:
            gamma = 1/sum_alpha
        else:
            gamma = npop
        fitval[:] = beta1*Fobj + beta2*(eta*FNv + gamma*np.sum(np.multiply(Fcon,alpha),axis=1))

    return fitval,cv,Nv

fitval,cv,Nv = evaluate_fitval(objval,conval)
print('fitness value:',fitval)
print('\nconstraint violation:',cv)
print('\nnumber of constraints violated:',Nv)


---
## Parental Selection
Parental selection is a process to select some individuals which are fit for mating and producing offspring. Some popular methods are tournament selection and roulette wheel. In this tutorial, we use the **binary tournament selection** method, which has the following procedure:
* select $k=2$ random individuals from the current parent population, and
* put the best individual in a mating pool.
The procedure stops when there are $N_{pop}$ individuals in the mating pool.

Note that this is a random process, that is, you will generate new mating pool each time you rerun the cell:


In [ ]:
#@title Parental Selection
def create_matingpool(population,fitval):
	npop,nvar = population.shape
	matingpool = np.zeros((npop,nvar))
	for kk in range(npop):
		ip1 = np.random.randint(npop)
		ip2 = np.random.randint(npop)
		if ip2 == ip1:
			while ip2 == ip1:
				ip2 = np.random.randint(npop)
		Ft1 = population[ip1-1,:]
		Ft2 = population[ip2-1,:]
		Fit1 = fitval[ip1-1]
		Fit2 = fitval[ip2-1]
		if Fit1 < Fit2:
			matingpool[kk,:] = Ft1
		else:
			matingpool[kk,:] = Ft2
	
	return matingpool

matingpool = create_matingpool(population,fitval)
print('matingpool:',matingpool)

---
## Crossover/ Recombination
Let's talk about the two main mechanisms that drive EA toward optimal solution, namely, **crossover** and **mutation**; we'll begin with the former.
* Crossover/ recombination is performed so that new individuals are expected to yield better fitness values than the old individuals. 
* In canonical EA, a pair of parents are usually selected from the mating pool to allow them producing a pair of offspring by a crossover method.
*This process is repeated until there is a population of offspring with size of $N_{pop}$. In this tutorial, we utilize **simulated binary crossover (SBX)**. Suppose $\boldsymbol{x}_1 = \{x_{1,1},\ldots,x_{d,1}\}$ and $\boldsymbol{x}_2 = \{x_{1,2},\ldots,x_{d,2}\}$ are a pair of parents, SBX generates a pair of offspring $\boldsymbol{c}_1 = \{c_{1,1},\ldots,c_{d,1}\}$ and $\boldsymbol{c}_2 = \{c_{1,2},\ldots,c_{d,2}\}$ from $\boldsymbol{x}_1$ and $\boldsymbol{x}_2$ with the following expression,
$$c_{j,1} = 0.5(x_{j,1}+x_{j,2}-\overline{\beta} |x_{j,2}-x_{j,1}|)$$
$$c_{j,2} = 0.5(x_{j,1}+x_{j,2}+\overline{\beta} |x_{j,2}-x_{j,1}|)$$
where
$$
	\overline{\beta} = \left\{
	\begin{array}{ll}
	(\alpha u)^{\frac{1}{\eta_c+1}}, & \text{if } u \leq \frac{1}{\alpha}, \\
	(\frac{1}{2-\alpha u})^{\frac{1}{\eta_c+1}}, & \text{otherwise}.
	\end{array}
	\right.
$$
where $\alpha = 2-\beta^{-(\eta_c+1)}$, and $\beta$ is calculated as follows,
$$
\beta = 1 + \frac{2}{x_{j,2}-x_{j,1}} \min(x_{j,1}-x_{j,L},x_{j,U}-x_{j,2}) \text{.}
$$
It is assumed that $x_{j,1} < x_{j,2}$ here. The symbols $u$ and $\eta_c$ denote a random value and distribution index, respectively. Here, we set $\eta_c=20$ in all experiment.

<br><br>
**Reference**:
Deb, Kalyanmoy, and Ram Bhushan Agrawal. "Simulated binary crossover for continuous search space." Complex systems 9, no. 2 (1995): 115-148.

The cell below executes a random crossover process to create a new population of offsprings:

In [ ]:
#@title Crossover
def SBX(indi1,indi2,nvar):
    nc = 20
    c1 = np.zeros(nvar)
    c2 = np.zeros(nvar)
    count = 0
    for i in range(nvar):
        nr = np.random.rand()
        if nr < 0.5:
            u = np.random.rand()
            sn = 1e-16
            if abs(indi1[i]-indi2[i]) > sn:
                if indi1[i] < indi2[i]:
                    p1 = indi1[i]
                    p2 = indi2[i]
                else:
                    p1 = indi2[i]
                    p2 = indi1[i]
                
                const1 = p1
                const2 = 1-p2
                
                beta1 = 1 + 2*const1/(p2-p1)
                beta2 = 1 + 2*const2/(p2-p1)
                alpha1 = 2 - beta1**(-(nc+1))
                alpha2 = 2 - beta2**(-(nc+1))
                
                if u <= (1/alpha1):
                    beta_l1 = (alpha1*u)**(1/(nc+1))
                else:
                    beta_l1 = (1/(2-alpha1*u))**(1/(nc+1))
                
                c1[i] = 0.5*((p1+p2)-beta_l1*abs(p2-p1))
                
                if u <= (1/alpha2):
                    beta_l2 = (alpha2*u)**(1/(nc+1))
                else:
                    beta_l2 = (1/(2-alpha2*u))**(1/(nc+1))
                
                c2[i] = 0.5*((p1+p2)+beta_l2*abs(p2-p1))

            else:
                c1[i] = indi1[i]
                c2[i] = indi2[i]
        else:
            c1[i] = indi1[i]
            c2[i] = indi2[i]
        if(c1[i] < 0 or c1[i] > 1) or (c2[i] < 0 or c2[i] > 1):
            count = count+1
        if(count > 0):
            print(count)

    c = [c1,c2]		
    return c

def crossover_process(matingpool,pcross):
    npop,nvar = matingpool.shape
    population = np.zeros((npop,nvar))
    for jj in range(0,npop,2):
        idx1 = np.random.randint(npop)
        idx2 = np.random.randint(npop)
        if idx2 == idx1:
            while idx2 == idx1:
                idx2 = np.random.randint(npop)
        n = np.random.rand()
        p1 = matingpool[idx1-1,:]
        p2 = matingpool[idx2-1,:]
        if(n < pcross):
            child = SBX(p1,p2,nvar)
            population[jj,:] = child[0]
            population[jj+1,:] = child[1]
        else:
            population[jj,:] = p1
            population[jj+1,:] = p2

    return population

print('Parent population:',population)
pcross = 0.9
population_cross = crossover_process(matingpool,pcross)
print('\nOffspring population:',population_cross)


---
## Mutation
* Mutation maintains the diversity of the population.
* In this tutorial, **polynomial mutation** is utilized, expressed as follows:
$$c_{j}^{'} = c_{j} + \overline{\delta} (x_{j,U}-x_{j,L})$$
where
$$
	\overline{\delta} = \left\{
	\begin{array}{ll}
	(2u+(1-2u)(1-\delta)^{\eta_m+1})^{\frac{1}{\eta_m+1}}-1, & \text{if } u \leq 0.5, \\
	1-(2(1-u)+2(u-0.5)(1-\delta)^{\eta_m+1})^{\frac{1}{\eta_m+1}}, & \text{otherwise}.
	\end{array}
	\right.
$$
where $\eta_m$ denotes the mutation distribution index. Here, we set the distribution index $\eta_m=20$ for all experiments.

<br><br>
**Reference**:
Deb, Kalyanmoy, and Debayan Deb. "Analysing mutation schemes for real-parameter genetic algorithms." International Journal of Artificial Intelligence and Soft Computing 4, no. 1 (2014): 1-28.

In [ ]:
#@title Mutation
def polymut(var,nvar,pmut):
    nm = 20
    mutchrom = var.copy()
    const = 1/(1+nm)
    for i in range(nvar):
        u = np.random.rand()
        rn = np.random.rand()
        if(rn < pmut):
            delta = min(var[i],1-var[i])
            if(u <= 0.5):
              delta2 = (2*u+(1-2*u)*(1-delta)**(nm+1))**const - 1
            else:
              delta2 = 1 - (2*(1-u)+2*(u-0.5)*(1-delta)**(nm+1))**const
            mutchrom[i] = mutchrom[i] + delta2
        if (mutchrom[i] < 0):
            mutchrom[i] = 0
        if (mutchrom[i] > 1):
            mutchrom[i] = 1
    return mutchrom

def mutation_process(population,pmut):
	npop,nvar = population.shape
	population_mut = population.copy()
	for kk in range(npop):
		var = population[kk,:]
		population_mut[kk,:] = polymut(var,nvar,pmut)
	
	return population_mut

print('Before mutation:',population_cross)
pmut = 0.8
population_mut = mutation_process(population_cross,pmut)
print('\nAfter mutation:',population_mut)

---
## Integrating Real-Coded GA Code
Finally, let's combine all of those EA mechanisms into a single integrated EA code! (let's call it ```soga```) For this demonstration , you can use either ackley or welded beam function. However, you can also define your own function too and try to optimize it by using this code.

You need to define the number of population (```npop```), maximum generation (```maxgen```), crossover probability (```pcross```), and mutation probability (```pmut```). Feel free to change these parameters and see how they will affect the optimization process.

Let's define the ```soga``` routine first by running the cell below:

In [ ]:
#@title Integrated RGA
def soga(fun_name,nvar,ncon,lb,ub,npop,maxgen,pcross,pmut,cht_type=None):	

    # Obtaining initial parent population (normalized form with value between 0 and 1)
    pop_parent = np.random.rand(npop,nvar)
      
    # Calculating responses (objective and constraint values) of parent population
    objval_parent,conval_parent = calc_obj_con(pop_parent,fun_name,ncon,lb,ub,cht_type)
      
    # Defining fitness value (can be real values, rank from MCR, etc) of parent poopulation
    fitval_parent,cv_parent,Nv_parent = evaluate_fitval(objval_parent,conval_parent,cht_type)

    # Finding feasible individuals of parent population
    ind_parent_feas = np.where(Nv_parent==0)[0]

    # Allocating lists for current best individual
    best_individual = []
    best_objval = []
    best_conval = []
    best_fitval = []

    if len(ind_parent_feas) > 0:
        pop_parent_feas = pop_parent[ind_parent_feas,:]
        objval_parent_feas = objval_parent[ind_parent_feas]
        conval_parent_feas = conval_parent[ind_parent_feas,:]
        fitval_parent_feas = fitval_parent[ind_parent_feas]

        # Sorting feasible individuals of parent population from lowest to highest fitness values
        allIndex = np.argsort(fitval_parent_feas)

        # Recording the current best individual
        best_individual.append(pop_parent_feas[allIndex[0],:])
        best_objval.append(objval_parent_feas[allIndex[0]])
        best_conval.append(conval_parent_feas[allIndex[0],:])
        best_fitval.append(fitval_parent_feas[allIndex[0]])
    else:
        # Sorting feasible individuals of parent population from lowest to highest fitness values
        allIndex = np.argsort(fitval_parent)

        # Recording the current best individual
        best_individual.append(np.nan*np.ones(nvar))
        best_objval.append(np.nan)
        best_conval.append(np.nan*np.ones(ncon))
        best_fitval.append(np.nan)

    # Generational looping
    for gen in range(maxgen):
        # if gen == 0:
        #     print('currently evaluating gen:',gen,', current optimum: ',objval_parent_feas[allIndex[0]])
        # else:
        #     print('currently evaluating gen:',gen,', current optimum: ',objval_child_feas[allIndex[0]])

        # Stopping criterion
        if gen == maxgen-1:
          break
        
        # Creating mating pool
        matingpool = create_matingpool(pop_parent,fitval_parent)
        
        # Conducting crossover
        pop_child = crossover_process(matingpool,pcross)
        
        # Conducting mutation
        pop_child_mut = mutation_process(pop_child,pmut)
        
        # Calculating responses (objective and constraint values) of child population
        objval_child,conval_child = calc_obj_con(pop_child_mut,fun_name,ncon,lb,ub,cht_type)

        # Elitism
        if len(ind_parent_feas) > 0:
            pop_child_mut[0,:] = best_individual[len(best_individual)-1]
            objval_child[0] = best_objval[len(best_objval)-1]
            conval_child[0,:] = best_conval[len(best_conval)-1]
        else:
            pop_child_mut[0,:] = pop_parent[allIndex[0],:]
            objval_child[0] = objval_parent[allIndex[0]]
            conval_child[0,:] = conval_parent[allIndex[0],:]

        # Defining fitness value (can be real values, rank from MCR, etc) of combined poopulation
        fitval_child,cv_child,Nv_child = evaluate_fitval(objval_child,conval_child,cht_type)

        # Finding feasible individuals of child population
        ind_child_feas = np.where(Nv_child==0)[0]

        if len(ind_child_feas) > 0:
            pop_child_feas = pop_child[ind_child_feas,:]
            objval_child_feas = objval_child[ind_child_feas]
            conval_child_feas = conval_child[ind_child_feas,:]
            fitval_child_feas = fitval_child[ind_child_feas]

            # Sorting individuals of combined population from lowest to highest fitness values
            allIndex = np.argsort(fitval_child_feas)

            # Recording the current best individual
            best_individual.append(pop_child_feas[allIndex[0],:])
            best_objval.append(objval_child_feas[allIndex[0]])
            best_conval.append(conval_child_feas[allIndex[0],:])
            best_fitval.append(fitval_child_feas[allIndex[0]])
        else:
            # Sorting individuals of combined population from lowest to highest fitness values
            allIndex = np.argsort(fitval_child)

            # Recording the current best individual
            best_individual.append(np.nan*np.ones(nvar))
            best_objval.append(np.nan)
            best_conval.append(np.nan*np.ones(ncon))
            best_fitval.append(np.nan)

        # New population
        pop_parent = pop_child_mut.copy()
        fitval_parent = fitval_child.copy()
        objval_parent = objval_child.copy()
        conval_parent = conval_child.copy()
        Nv_parent = Nv_child.copy()

        # Finding feasible individuals of parent population
        ind_parent_feas = np.where(Nv_parent==0)[0]

    best_individual = np.array(best_individual)
    best_objval = np.array(best_objval)
    best_conval = np.array(best_conval)

    result = [best_objval,best_individual,best_conval]
    return result


Let's try it for solving the Ackley and the Welded beam problem (please change ```fun_name```):

Note: for the Ackley problem, you can decrease or increase the number of variables (```nvar```) to make it less or more difficult, respectively. Further, you can change the population size (```npop```), maximum generations(```maxgen```), crossover probability (```pcross```) and mutation probability(```pmut```). We suggest you to play around with these parameters and see how they affect the optimization process.

In [ ]:
#@title Running RGA
#=========================================================================
# GA parameters and search space

fun_name = 'ackley'
# fun_name = 'weldedbeam'

npop = 100 # Population size
maxgen = 100 # maximum generations
pcross = 0.9 # Crossover probability
pmut = 0.1 # Mutation probability
	
if fun_name == 'ackley':
    nvar = 2
    ncon = 0
    cht_types = None
    lb = -32.768*np.ones(nvar)
    ub = 32.768*np.ones(nvar)
elif fun_name == 'weldedbeam':
    nvar = 4
    ncon = 6
    #cht_types = 'SoF'
    cht_types = 'G-MCR'
    lb = np.array([0.125,0.1,0.1,0.1])
    ub = np.array([2,10,10,2])

# ========================================================================
# Optimization - Trial
result = soga(fun_name,nvar,ncon,lb,ub,npop,maxgen,pcross,pmut,cht_type=cht_types)
best_objval = result[0]
best_individual = result[1]
best_conval = result[2]

print('Objective value of the best solution: ',best_objval[len(best_objval)-1])
print('Decision variables of the best solution: ',best_individual[len(best_objval)-1])
print('Constraint values of the best solution: ',best_conval[len(best_objval)-1])

# ========================================================================
# Plotting the convergence performance
fig = plt.figure()
plt.plot(np.arange(0,maxgen),best_objval)
plt.xlabel('Generation')
plt.ylabel('Obtained Minimum Value')
plt.grid()
plt.show()

You should see the obtained minimum value decreases as the number of generation increases, which indicates the success of the EA search! Maybe you also want to try experimenting with the parameters or change the CHT. Note that you can also increase the dimensionality of the Ackley function to make the problem harder (and how would you solve that? What parameters that you need to change?).

---
# Statictical Analysis with Box Plot

Especially when you do research in the field of EA, you need to perform statistical analysis to compare the performance of various EA methods (or different crossover techniques, let's say). By that, we mean that you should run EA multiple times because of the random nature of EA. What you want to analyze is then the average of the performance of EA, not its performance in a single run. Notice that you need to increase the number of trials to obtain a more statistically valid result. However, remember that increasing the number of trials also increases the computation time.

* As explained in the theory overview, stochastic optimization methods such as EA require statistical analysis to ensure their consistency in terms of performance. 
* In this tutorial, we will conduct the statistical analysis visually with a box plot. Remember that usually we need to do a statistical hypothesis test to further verify the performance.
* As EA requires some user-defined settings, let's check its performance sensitivity by changing the basic parameters!


First, let's try varying the population size. How would different population sizes affect the EA search? With a fixed number of maximum evals, should we (1) deploy a lower population size with more generations, or (2) higher population size with lower generations?. Here, you can perform the experiment by changing the following list, e.g. ```npops = [300,30]```, which means that you will experiment with $N_{pop}=300$ and $N_{pop}=30$. 

In [ ]:
#@title Statistical Analysis - Npop
# ========================================================================
# Setting number of trials
n_try = 5 # Number of trials (it takes time for more trials)

#=========================================================================
# Optimization - Npop Sensitivity

#fun_name = 'ackley'
fun_name = 'weldedbeam'

if fun_name == 'ackley':
    nvar = 2
    ncon = 0
    cht_types = None
    lb = -32.768*np.ones(nvar)
    ub = 32.768*np.ones(nvar)
elif fun_name == 'weldedbeam':
    nvar = 4
    ncon = 6
    #cht_types = 'SoF'
    cht_types = 'G-MCR'
    lb = np.array([0.125,0.1,0.1,0.1])
    ub = np.array([2,10,10,2])

npops = [300,30] # Please make the list of various number of populations here
maxeval = 30000 # The number of maximum function evaluations, your number of generation is then maxeval/npop
pcross = 0.9 # Crossover probability
pmut = 0.1 # Mutation probability

collect_all_objval = []
for i_pop in range(len(npops)):
    npop = npops[i_pop]
    maxgen = int(maxeval/npop)

    all_objval = np.zeros(n_try)
    for i_try in range(n_try):
        
        print('currently running npop: ',npop,' run: ',i_try)
        
        result = soga(fun_name,nvar,ncon,lb,ub,npop,maxgen,pcross,pmut,cht_type=cht_types)
        best_objval = result[0]
        best_individual = result[1]
        best_conval = result[2]
        
        all_objval[i_try] = best_objval[len(best_objval)-1]
      
    collect_all_objval.append(all_objval)

# Box plot of Population Size Sensitivity	
fig,ax = plt.subplots()
ax.boxplot(collect_all_objval)
plt.xlabel('Population Size')
plt.ylabel('Optimum Objective Value')
plt.xticks(range(1,len(npops)+1),npops)
ax.set_xticklabels(npops, rotation=45, ha='right')
plt.grid(which='major')
plt.grid(which='minor',linestyle='-.',alpha=0.2)
plt.minorticks_on()
plt.tight_layout()

Let's try varying the crossover probability. How would different crossover probability affect the EA search? Should we set a high or maybe a lower value of crossover probability? Do the experiment by changing the values of the crossover probability in ```pcrosses```, which is a list, e.g. ```npops = [0.9,0.1]```, which means that you will perform experiment with $P_{cross}=0.9$ and $P_{cross}=0.1$. 

In [ ]:
#@title Statistical Analysis - Pcross
#=========================================================================
# Setting number of trials
n_try = 5

#=========================================================================
# Optimization - Pcross Sensitivity
fun_name = 'ackley'
#fun_name = 'weldedbeam'

if fun_name == 'ackley':
    nvar = 2
    ncon = 0
    cht_types = None
    lb = -32.768*np.ones(nvar)
    ub = 32.768*np.ones(nvar)
elif fun_name == 'weldedbeam':
    nvar = 4
    ncon = 6
    cht_types = 'SoF'
    lb = np.array([0.125,0.1,0.1,0.1])
    ub = np.array([2,10,10,2])

npop = 100 # Population size
maxgen = 300 # Number of maximum generations
pcrosses = [0.9,0.1] # Please make the list of various crossover probability here
pmut = 0.1 # Mutation probability

collect_all_objval2 = []
for i_cross in range(len(pcrosses)):
	pcross = pcrosses[i_cross]
	all_objval2 = np.zeros(n_try)
	for i_try in range(n_try):
		
		print('currently running pcross: ',pcross,' run: ',i_try)
		
		result = soga(fun_name,nvar,ncon,lb,ub,npop,maxgen,pcross,pmut,cht_type=cht_types)
		best_objval = result[0]
		best_individual = result[1]
		best_conval = result[2]
		
		all_objval2[i_try] = best_objval[len(best_objval)-1]
		
	collect_all_objval2.append(all_objval2)

# Box plot of Crossover Probability Sensitivity	
fig,ax = plt.subplots()
ax.boxplot(collect_all_objval2)
plt.xlabel('Crossover Probability')
plt.ylabel('Optimum Objective Value')
plt.xticks(range(1,len(pcrosses)+1),pcrosses)
ax.set_xticklabels(pcrosses, rotation=45, ha='right')
plt.grid(which='major')
plt.grid(which='minor',linestyle='-.',alpha=0.2)
plt.minorticks_on()
plt.tight_layout()


Next, let's try varying the mutation probability. How would different mutation probabilities affect the EA search? What would happen if we set a mutation probability that is too high or maybe too low? Experiment by changing the values of the mutation probability in ```pmuts```, which is a list, e.g. ```npops = [0.9,0.1]```, which means that you will perform experiment with $P_{mut}=0.9$ and $P_{mut}=0.1$. 

In [ ]:
#@title Statistical Analysis - Pmut
#=========================================================================
# Setting number of trials
n_try = 5

#=========================================================================
# Optimization - Pmut Sensitivity
fun_name = 'ackley'
#fun_name = 'weldedbeam'

if fun_name == 'ackley':
    nvar = 2
    ncon = 0
    cht_types = None
    lb = -32.768*np.ones(nvar)
    ub = 32.768*np.ones(nvar)
elif fun_name == 'weldedbeam':
    nvar = 4
    ncon = 6
    cht_types = 'SoF'
    lb = np.array([0.125,0.1,0.1,0.1])
    ub = np.array([2,10,10,2])

npop = 100 # Population size
maxgen = 300 # Maximum generation
pcross = 0.9 # Crossover probability
pmuts = [0.9,0.1] # Please make the list of various mutation probability here

collect_all_objval3 = []
for i_mut in range(len(pmuts)):
	pmut = pmuts[i_mut]
	all_objval3 = np.zeros(n_try)
	for i_try in range(n_try):
		
		print('currently running pmut: ',pmut,' run: ',i_try)
		
		result = soga(fun_name,nvar,ncon,lb,ub,npop,maxgen,pcross,pmut,cht_type=cht_types)
		best_objval = result[0]
		best_individual = result[1]
		best_conval = result[2]
		
		all_objval3[i_try] = best_objval[len(best_objval)-1]
		
	collect_all_objval3.append(all_objval3)

# Box plot of Mutation Probability Sensitivity
fig2,ax2 = plt.subplots()
ax2.boxplot(collect_all_objval3)
plt.xlabel('Mutation Probability')
plt.ylabel('Optimum Objective Value')
plt.xticks(range(1,len(pmuts)+1),pmuts)
ax2.set_xticklabels(pmuts, rotation=45, ha='right')
plt.grid(which='major')
plt.grid(which='minor',linestyle='-.',alpha=0.2)
plt.minorticks_on()
plt.tight_layout()
plt.show()

Finally, let's study the effect of constraint handling. We will compare the conventional SoF method with G-MCR, the latter is one of state-of-the-art CHT techniques.

In [ ]:
#@title Statistical Analysis - CHT
# ========================================================================
# Setting number of trials
n_try = 5

#=========================================================================
# Optimization - CHT Comparison


fun_name = 'weldedbeam'
nvar = 4
ncon = 6
lb = np.array([0.125,0.1,0.1,0.1])
ub = np.array([2,10,10,2])

cht_types = ['SoF','G-MCR']
npop = 30
maxeval = 30000
pcross = 0.9
pmut = 0.1

collect_all_objval = []
for i_cht in range(len(cht_types)):
    cht_type = cht_types[i_cht]
    maxgen = int(maxeval/npop)

    all_objval = np.zeros(n_try)
    for i_try in range(n_try):
        
        print('currently running CHT: ',cht_type,' run: ',i_try)
        
        result = soga(fun_name,nvar,ncon,lb,ub,npop,maxgen,pcross,pmut,cht_type=cht_type)
        best_objval = result[0]
        best_individual = result[1]
        best_conval = result[2]
        
        all_objval[i_try] = best_objval[len(best_objval)-1]
      
    collect_all_objval.append(all_objval)

# Box plot of Population Size Sensitivity	
fig,ax = plt.subplots()
ax.boxplot(collect_all_objval)
plt.xlabel('Population Size')
plt.ylabel('Optimum Objective Value')
plt.xticks(range(1,len(cht_types)+1),cht_types)
ax.set_xticklabels(cht_types, rotation=45, ha='right')
plt.grid(which='major')
plt.grid(which='minor',linestyle='-.',alpha=0.2)
plt.minorticks_on()
plt.tight_layout()